In [1]:
import bambi as bmb
import numpy as np
import pandas as pd

In [2]:
#%load_ext autoreload
#%autoreload 2

In [3]:
data = bmb.load_data("sleepstudy")

In [4]:
data.head()

,Reaction,Days,Subject
0,249.5600,0,308
1,258.7047,1,308
2,250.8006,2,308
3,321.4398,3,308
4,356.8519,4,308


In [5]:
model = bmb.Model("Reaction ~ 1 + Days + (1 + Days | Subject)", data)
model

       Formula: Reaction ~ 1 + Days + (1 + Days | Subject)
        Family: gaussian
          Link: mu = identity
  Observations: 180
        Priors: 
    target = mu
        Common-level effects
            Intercept ~ Normal(mu: 298.5079, sigma: 261.0092)
            Days ~ Normal(mu: 0.0, sigma: 48.8915)
        
        Group-level effects
            1|Subject ~ Normal(mu: 0.0, sigma: HalfNormal(sigma: 261.0092))
            Days|Subject ~ Normal(mu: 0.0, sigma: HalfNormal(sigma: 48.8915))
        
        Auxiliary parameters
            sigma ~ HalfStudentT(nu: 4.0, sigma: 56.1721)

In [6]:
idata = model.fit()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [Reaction_sigma, Intercept, Days, 1|Subject_sigma, 1|Subject_offset, Days|Subject_sigma, Days|Subject_offset]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 17 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics


**Problema:** No estamos guardando la correlacion entre los parametros. No podriamos generar muestras con el metodo "gaussian" porque estariamos omitiendo la correlacion.

Una alternativa seria computar esa correlacion...

In [7]:
import formulae
from formulae import design_matrices

formulae.config["EVAL_UNSEEN_CATEGORIES"] = "silent"

dm = design_matrices("Reaction ~ 1 + Days + (1 + Days|Subject)", data)

df_new = data.head(10).reset_index(drop=True)
df_new["Subject"] = "xxx"
df_new = pd.concat([df_new, data.head(10)])
df_new = df_new.reset_index(drop=True)

dm_new = dm.group.evaluate_new_data(df_new)
dm_new

GroupEffectsMatrix with shape (20, 38)
Terms:  
  1|Subject  
    kind: intercept
    groups: ['308', '309', '310', '330', '331', '332', '333', '334', '335', '337', '349', '350',
      '351', '352', '369', '370', '371', '372', '__NEW_FACTOR_GROUP__']
    columns: 0:19
  Days|Subject  
    kind: numeric
    groups: ['308', '309', '310', '330', '331', '332', '333', '334', '335', '337', '349', '350',
      '351', '352', '369', '370', '371', '372', '__NEW_FACTOR_GROUP__']
    columns: 19:38

To access the actual design matrix do 'np.array(this_obj)'

In [8]:
import xarray as xr

In [9]:
factors_with_new_levels = dm_new.factors_with_new_levels

In [10]:
model.response_component.group_specific_groups

{'Subject': ['1|Subject', 'Days|Subject']}

In [11]:
ndraws = len(idata.posterior.coords["draw"])
factor_dim = "Subject__factor_dim"

factor_levels = idata.posterior.coords[factor_dim].to_numpy()
factor_sampled_idxs = np.random.choice(np.arange(len(factor_levels)), size=ndraws)

original_draws_1 = idata.posterior["1|Subject"].to_numpy()
new_group_draws_1 = original_draws_1[:, np.arange(ndraws), factor_sampled_idxs]

original_draws_2 = idata.posterior["Days|Subject"].to_numpy()
new_group_draws_2 = original_draws_2[:, np.arange(ndraws), factor_sampled_idxs]

coords = {
    "chain": np.arange(2), 
    "draw": np.arange(ndraws), 
    "Subject__factor_dim": ["__NEW_FACTOR_GROUP__"]
}
new_group_draws_1 = xr.DataArray(new_group_draws_1[..., np.newaxis], coords=coords)
new_group_draws_2 = xr.DataArray(new_group_draws_2[..., np.newaxis], coords=coords)

In [12]:
z1 = xr.concat([idata.posterior["1|Subject"], new_group_draws_1], dim="Subject__factor_dim")
z2 = xr.concat([idata.posterior["Days|Subject"], new_group_draws_2], dim="Subject__factor_dim")

u = xr.Dataset({"1|Subject": z1, "Days|Subject": z2})
u = u.to_stacked_array("__variables__", ("chain", "draw"))

In [13]:
response_dim = "Reaction_obs"
design_matrix_dims = (response_dim, "__variables__")

Z = np.asarray(dm_new)
Z = xr.DataArray(Z, dims=design_matrix_dims)
xr.dot(Z, u)

<xarray.DataArray (Reaction_obs: 20, chain: 2, draw: 1000)>
array([[[ 23.00057335,  35.39041776,  23.68888148, ...,  -3.04741072,
          -2.39313375, -56.26629592],
        [ 20.10735357,  17.3119835 ,  26.87686572, ...,  11.64314283,
         -10.58335849, -60.42401618]],

       [[ 28.31334024,  31.94726195,  21.69053368, ...,  -7.93186741,
          -4.25037768, -58.98639359],
        [ 31.99096578,  16.84154176,  21.40522276, ...,  12.37132474,
          -6.90206758, -65.08265469]],

       [[ 33.62610714,  28.50410613,  19.69218588, ..., -12.81632409,
          -6.10762161, -61.70649126],
        [ 43.874578  ,  16.37110002,  15.93357981, ...,  13.09950665,
          -3.22077666, -69.7412932 ]],

       ...,

       [[ 60.18994161,  78.18949671,  62.04540784, ...,  46.99769479,
          69.26949598,  78.43407027],
        [103.29263907,  72.24535217,  38.11966965, ...,  68.2068358 ,
         100.71937229,  50.82500918]],

       [[ 65.50270851,  91.76691803,  69.08052278, ...,  55.65157231,
          76.99268626,  89.14640778],
        [115.17625128,  84.03873892,  43.42771177, ...,  79.27169534,
         112.59355723,  57.84840865]],

       [[ 70.8154754 , 105.34433934,  76.11563773, ...,  64.30544982,
          84.71587653,  99.85874528],
        [127.05986349,  95.83212567,  48.73575388, ...,  90.33655488,
         124.46774216,  64.87180812]]])
Coordinates:
  * chain    (chain) int64 0 1
  * draw     (draw) int64 0 1 2 3 4 5 6 7 8 ... 992 993 994 995 996 997 998 999
Dimensions without coordinates: Reaction_obs

In [14]:
idata.posterior.coords

Coordinates:
  * chain                (chain) int64 0 1
  * draw                 (draw) int64 0 1 2 3 4 5 6 ... 994 995 996 997 998 999
  * Subject__factor_dim  (Subject__factor_dim) <U3 '308' '309' ... '371' '372'

In [19]:
df_new

,Reaction,Days,Subject
0,249.5600,0,xxx
1,258.7047,1,xxx
2,250.8006,2,xxx
3,321.4398,3,xxx
4,356.8519,4,xxx
5,414.6901,5,xxx
6,382.2038,6,xxx
7,290.1486,7,xxx
8,430.5853,8,xxx
9,466.3535,9,xxx


In [22]:
p = model.predict(idata, data=df_new, inplace=False)

In [28]:
p.posterior["Reaction_mean"]

<xarray.DataArray 'Reaction_mean' (chain: 2, draw: 1000, Reaction_obs: 20)>
array([[[288.91798515, 302.83855915, 316.75913315, ..., 388.93205307,
         405.51840428, 422.10475549],
        [262.27414249, 268.48685075, 274.69955902, ..., 399.85349031,
         423.08677571, 446.32006111],
        [271.27298659, 282.88586033, 294.49873408, ..., 397.03602649,
         415.87691741, 434.71780832],
        ...,
        [264.71078261, 269.03051202, 273.35024143, ..., 386.96458586,
         407.30932781, 427.65406976],
        [258.82430094, 275.62323836, 292.42217578, ..., 404.41944945,
         424.66137725, 444.90330504],
        [280.34740392, 282.96964869, 285.59189347, ..., 402.30664633,
         423.02600658, 443.74536684]],

       [[287.15499731, 300.51306902, 313.87114072, ..., 411.99803072,
         432.16048665, 452.32294259],
        [268.96787395, 279.68764266, 290.40741138, ..., 389.78614743,
         412.76974464, 435.75334184],
        [275.94570784, 290.09868501, 304.25166218, ..., 378.96624413,
         396.25764052, 413.5490369 ],
        ...,
        [243.58396943, 253.75715339, 263.93033735, ..., 378.02754194,
         398.81226438, 419.59698681],
        [277.72145054, 293.28720895, 308.85296737, ..., 412.5641983 ,
         433.28655587, 454.00891343],
        [250.87484153, 249.29065745, 247.70647336, ..., 390.17759468,
         410.60374143, 431.02988818]]])
Coordinates:
  * chain         (chain) int64 0 1
  * draw          (draw) int64 0 1 2 3 4 5 6 7 ... 993 994 995 996 997 998 999
  * Reaction_obs  (Reaction_obs) int64 0 1 2 3 4 5 6 7 ... 13 14 15 16 17 18 19

Necesitamos otros cambios a nivel de formulae

* Si hay observaciones nuevas, se agrega un nuevo grupo, a la derecha de los grupos existentes.
    * Todos los nuevos grupos son interpretados como UN UNICO nuevo grupo
    * No va a tener sentido para los efectos fijos
    * Si va a tener sentido para los efectos aleatorios
* Será mas fácil cuando se tengan que hacer predicciones para un nuevo grupo.
    * Simplemente "apendeamos" 